# **Hyperparameter Tuning Using Optuna For Classification Task**

# **🔧 Install Required Packages**

In [1]:
# %pip install optuna
# %pip install optuna-integration[xgboost]
# %pip install plotly

# **📥 Load and Prepare the Pima Indian Diabetes Dataset**

In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd


# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"


columns = [
    'Pregnancies',
    'Glucose',
    'BloodPressure',
    'SkinThickness',
    'Insulin',
    'BMI',
    'DiabetesPedigreeFunction',
    'Age',
    'Outcome'
]


# Load the dataset
df = pd.read_csv(url, names=columns)


df.head()

c:\Users\avina\Documents\GitHub\PyTorch\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


# **🧹 Data Cleaning and Imputation**

In [3]:
import numpy as np


# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = [
    'Glucose',
    'BloodPressure',
    'SkinThickness',
    'Insulin',
    'BMI'
]


df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)


# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)


# Check if there are any remaining missing values
print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


# **✂️ Split Data into Train and Test Sets & Scale**

In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']


# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42
)


# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')

Training set shape: (537, 8)
Test set shape: (231, 8)


# **🌲 Define Optuna Objective Function for RandomForest**

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(
        model,
        X_train,
        y_train,
        cv=3,
        scoring='accuracy'
    ).mean()

    return score  # Return the accuracy score for Optuna to maximize

# **🔎 Run Optuna Study with TPE Sampler**

In [6]:
# Create a study object and optimize the objective function
# We aim to maximize accuracy
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler()
)


# Run 50 trials to find the best hyperparameters
study.optimize(objective, n_trials=50)

[I 2025-09-13 03:39:46,582] A new study created in memory with name: no-name-0c4b77de-5ffa-4e75-97a0-c60ea2055ea6
[I 2025-09-13 03:39:47,433] Trial 0 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 103, 'max_depth': 17}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-09-13 03:39:48,659] Trial 1 finished with value: 0.7802607076350093 and parameters: {'n_estimators': 133, 'max_depth': 16}. Best is trial 1 with value: 0.7802607076350093.
[I 2025-09-13 03:39:49,222] Trial 2 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 60, 'max_depth': 19}. Best is trial 1 with value: 0.7802607076350093.
[I 2025-09-13 03:39:49,691] Trial 3 finished with value: 0.7597765363128492 and parameters: {'n_estimators': 55, 'max_depth': 12}. Best is trial 1 with value: 0.7802607076350093.
[I 2025-09-13 03:39:51,113] Trial 4 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 198, 'max_depth': 3}. Best is trial 1 with value: 0.7802607

# **🏆 Show Best Hyperparameters from Optuna**

In [7]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7839851024208566
Best hyperparameters: {'n_estimators': 73, 'max_depth': 18}


# **🧪 Evaluate Best Model on Test Set**

In [9]:
from sklearn.metrics import accuracy_score


# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)


# Fit the model to the training data
best_model.fit(X_train, y_train)


# Make predictions on the test set
y_pred = best_model.predict(X_test)


# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)


# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.76


# **🌱 Samplers in Optuna - Random Sampler Example**

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)


    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )


    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(
        model,
        X_train,
        y_train,
        cv=3,
        scoring='accuracy'
    ).mean()


    return score  # Return the accuracy score for Optuna to maximize

In [10]:
# We aim to maximize accuracy
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.RandomSampler()
)


# Run 50 trials to find the best hyperparameters
study.optimize(objective, n_trials=50)

[I 2025-09-13 01:27:23,399] A new study created in memory with name: no-name-a85b0402-3a80-4d0b-84ed-bc6b5736f56a
[I 2025-09-13 01:27:24,654] Trial 0 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 168, 'max_depth': 18}. Best is trial 0 with value: 0.7709497206703911.
[I 2025-09-13 01:27:25,454] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 82, 'max_depth': 15}. Best is trial 0 with value: 0.7709497206703911.
[I 2025-09-13 01:27:26,480] Trial 2 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 133, 'max_depth': 5}. Best is trial 0 with value: 0.7709497206703911.
[I 2025-09-13 01:27:27,645] Trial 3 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 154, 'max_depth': 16}. Best is trial 3 with value: 0.7746741154562384.
[I 2025-09-13 01:27:28,448] Trial 4 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 110, 'max_depth': 12}. Best is trial 3 with value: 0.774674

In [11]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 118, 'max_depth': 16}


In [12]:
from sklearn.metrics import accuracy_score


# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)


# Fit the model to the training data
best_model.fit(X_train, y_train)


# Make predictions on the test set
y_pred = best_model.predict(X_test)


# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)


# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.74


# **🔲 Grid Sampler Example**

In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.GridSampler(search_space)
)


study.optimize(objective)

[I 2025-09-13 01:28:10,314] A new study created in memory with name: no-name-9336acf1-c8b7-4f9d-8694-2c395928677d
[I 2025-09-13 01:28:10,929] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-13 01:28:11,819] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-13 01:28:12,141] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-13 01:28:12,772] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-13 01:28:13,384] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [15]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score


# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)


# Fit the model to the training data
best_model.fit(X_train, y_train)


# Make predictions on the test set
y_pred = best_model.predict(X_test)


# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)


# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.74


# **📊 Visualize Optuna Study Results**

In [10]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

# **📈 Optimization History Plot**

In [11]:
# 1. Optimization History
plot_optimization_history(study).show()

# **📉 Parallel Coordinates Plot**

In [12]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

# **📊 Slice Plot**

In [13]:
# 3. Slice Plot
plot_slice(study).show()

# **🗺️ Contour Plot**

In [14]:
# 4. Contour Plot
plot_contour(study).show()

# **⭐ Hyperparameter Importance Plot**

In [15]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

# **🤖 Optimize Multiple Classifiers with Optuna**

In [23]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [ ]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical(
        'classifier',
        ['SVM', 'RandomForest', 'GradientBoosting']
    )

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)


        kernel = trial.suggest_categorical(
            'kernel',
            ['linear', 'rbf', 'poly', 'sigmoid']
        )

        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(
            C=c, 
            kernel=kernel, 
            gamma=gamma, 
            random_state=42
        )
        

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )
        

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )



    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(
        model,
        X_train,
        y_train,
        cv=3,
        scoring='accuracy'
    ).mean()


    return score

In [25]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-09-13 01:28:32,122] A new study created in memory with name: no-name-a80fe11e-19ed-478f-aac7-ae2d0e3b0b2d
[I 2025-09-13 01:28:35,246] Trial 0 finished with value: 0.7690875232774674 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 126, 'learning_rate': 0.01886066265895101, 'max_depth': 5, 'min_samples_split': 3, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-13 01:28:37,151] Trial 1 finished with value: 0.7597765363128491 and parameters: {'classifier': 'RandomForest', 'n_estimators': 176, 'max_depth': 19, 'min_samples_split': 3, 'min_samples_leaf': 9, 'bootstrap': True}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-13 01:28:38,457] Trial 2 finished with value: 0.7783985102420857 and parameters: {'classifier': 'RandomForest', 'n_estimators': 153, 'max_depth': 13, 'min_samples_split': 2, 'min_samples_leaf': 9, 'bootstrap': False}. Best is trial 2 with value: 0.7783985102420857.
[I 2025-09-13 01:28:38,529] Trial 3

In [26]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.1348000558858413, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [27]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.769088,2025-09-13 01:28:32.126140,2025-09-13 01:28:35.246851,0 days 00:00:03.120711,NaN,NaN,GradientBoosting,NaN,NaN,0.018861,5.0,2.0,3.0,126.0,COMPLETE
1,1,0.759777,2025-09-13 01:28:35.248231,2025-09-13 01:28:37.151881,0 days 00:00:01.903650,NaN,True,RandomForest,NaN,NaN,NaN,19.0,9.0,3.0,176.0,COMPLETE
2,2,0.778399,2025-09-13 01:28:37.152882,2025-09-13 01:28:38.457833,0 days 00:00:01.304951,NaN,False,RandomForest,NaN,NaN,NaN,13.0,9.0,2.0,153.0,COMPLETE
3,3,0.698324,2025-09-13 01:28:38.460749,2025-09-13 01:28:38.529387,0 days 00:00:00.068638,18.938806,NaN,SVM,scale,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
4,4,0.776536,2025-09-13 01:28:38.530385,2025-09-13 01:28:41.717049,0 days 00:00:03.186664,NaN,NaN,GradientBoosting,NaN,NaN,0.013236,7.0,9.0,10.0,199.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.703911,2025-09-13 01:29:58.750007,2025-09-13 01:29:58.790240,0 days 00:00:00.040233,0.100408,NaN,SVM,auto,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.785847,2025-09-13 01:29:58.791240,2025-09-13 01:29:58.825933,0 days 00:00:00.034693,0.211270,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.783985,2025-09-13 01:29:58.826963,2025-09-13 01:29:58.865188,0 days 00:00:00.038225,0.424653,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.743017,2025-09-13 01:29:58.865188,2025-09-13 01:30:03.560233,0 days 00:00:04.695045,NaN,NaN,GradientBoosting,NaN,NaN,0.160373,20.0,2.0,4.0,293.0,COMPLETE


In [28]:
study.trials_dataframe()['params_classifier'].value_counts()

params_classifier
SVM                 71
RandomForest        16
GradientBoosting    13
Name: count, dtype: int64

In [29]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

params_classifier
GradientBoosting    0.743303
RandomForest        0.767691
SVM                 0.775645
Name: value, dtype: float64

In [30]:
# 1. Optimization History
plot_optimization_history(study).show()

In [31]:
# 3. Slice Plot
plot_slice(study).show()

In [32]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

# **🌸 XGBoost Hyperparameter Tuning with Optuna and Pruning**

In [33]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np


# Load the Iris dataset
X, y = load_iris(return_X_y=True)


# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)


# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(
        trial,
        "eval-mlogloss"
    )  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        # Ensure the eval datasets and names are specified
        evals=[(dtrain, "train"), (dtest, "eval")],
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy


# Create a study with pruning
study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.SuccessiveHalvingPruner()
)

study.optimize(objective, n_trials=50)


# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")

[I 2025-09-13 01:30:10,853] A new study created in memory with name: no-name-3b225230-a86d-4df0-b0a0-c0c784635255


[0]	train-mlogloss:0.99671	eval-mlogloss:0.99266
[1]	train-mlogloss:0.94909	eval-mlogloss:0.95156
[2]	train-mlogloss:0.89149	eval-mlogloss:0.90085
[3]	train-mlogloss:0.80654	eval-mlogloss:0.80904
[4]	train-mlogloss:0.77578	eval-mlogloss:0.78480
[5]	train-mlogloss:0.72967	eval-mlogloss:0.74295
[6]	train-mlogloss:0.66522	eval-mlogloss:0.67011
[7]	train-mlogloss:0.60242	eval-mlogloss:0.60111
[8]	train-mlogloss:0.57803	eval-mlogloss:0.58014
[9]	train-mlogloss:0.52533	eval-mlogloss:0.52216
[10]	train-mlogloss:0.48820	eval-mlogloss:0.48207
[11]	train-mlogloss:0.47444	eval-mlogloss:0.47361
[12]	train-mlogloss:0.44098	eval-mlogloss:0.43821
[13]	train-mlogloss:0.41538	eval-mlogloss:0.41041
[14]	train-mlogloss:0.38255	eval-mlogloss:0.37097
[15]	train-mlogloss:0.36603	eval-mlogloss:0.35588
[16]	train-mlogloss:0.35436	eval-mlogloss:0.34303
[17]	train-mlogloss:0.33395	eval-mlogloss:0.32338
[18]	train-mlogloss:0.31543	eval-mlogloss:0.30317
[19]	train-mlogloss:0.31127	eval-mlogloss:0.30062
[20]	train

[I 2025-09-13 01:30:12,426] Trial 0 finished with value: 1.0 and parameters: {'lambda': 8.361707558495242e-05, 'alpha': 0.06196160979897864, 'eta': 0.10291141872374487, 'gamma': 1.513565094933609e-05, 'max_depth': 4, 'min_child_weight': 4, 'subsample': 0.8013333139575807, 'colsample_bytree': 0.4242714468857873}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.98798	eval-mlogloss:0.98292
[1]	train-mlogloss:0.93161	eval-mlogloss:0.92798
[2]	train-mlogloss:0.85044	eval-mlogloss:0.84298
[3]	train-mlogloss:0.77249	eval-mlogloss:0.76314
[4]	train-mlogloss:0.70536	eval-mlogloss:0.69193
[5]	train-mlogloss:0.64524	eval-mlogloss:0.62783
[6]	train-mlogloss:0.59361	eval-mlogloss:0.57214
[7]	train-mlogloss:0.54652	eval-mlogloss:0.52266
[8]	train-mlogloss:0.50495	eval-mlogloss:0.47747
[9]	train-mlogloss:0.46714	eval-mlogloss:0.43695
[10]	train-mlogloss:0.43649	eval-mlogloss:0.40691
[11]	train-mlogloss:0.40504	eval-mlogloss:0.37401
[12]	train-mlogloss:0.38069	eval-mlogloss:0.34833
[13]	train-mlogloss:0.35497	eval-mlogloss:0.32054
[14]	train-mlogloss:0.33219	eval-mlogloss:0.29439
[15]	train-mlogloss:0.31515	eval-mlogloss:0.27753
[16]	train-mlogloss:0.29806	eval-mlogloss:0.26067
[17]	train-mlogloss:0.28006	eval-mlogloss:0.24182
[18]	train-mlogloss:0.26700	eval-mlogloss:0.22804
[19]	train-mlogloss:0.25308	eval-mlogloss:0.21484
[20]	train

[I 2025-09-13 01:30:13,838] Trial 1 finished with value: 1.0 and parameters: {'lambda': 5.107038482748965e-08, 'alpha': 1.003238074601496e-05, 'eta': 0.08648103657018406, 'gamma': 1.5152022940686848e-06, 'max_depth': 3, 'min_child_weight': 4, 'subsample': 0.8136426215577014, 'colsample_bytree': 0.5565756914270521}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:1.04288	eval-mlogloss:1.03964
[1]	train-mlogloss:1.01308	eval-mlogloss:1.00756


[I 2025-09-13 01:30:33,566] Trial 48 pruned. Trial was pruned at iteration 2.


[0]	train-mlogloss:0.94403	eval-mlogloss:0.93628
[1]	train-mlogloss:0.87346	eval-mlogloss:0.86175


[I 2025-09-13 01:30:33,639] Trial 49 pruned. Trial was pruned at iteration 2.


Best trial: {'lambda': 8.361707558495242e-05, 'alpha': 0.06196160979897864, 'eta': 0.10291141872374487, 'gamma': 1.513565094933609e-05, 'max_depth': 4, 'min_child_weight': 4, 'subsample': 0.8013333139575807, 'colsample_bytree': 0.4242714468857873}
Best accuracy: 1.0


# **📈 Visualize XGBoost Intermediate Values**

In [34]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()